Una volta estratti i parametri modificabili all'interno del NN, possiamo ora procedere con il meccanismo di gradient descent al fine di aggiustare questi parametri al fine di minimizzare la loss function

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random

%matplotlib inline

In [5]:
class Value:
    
    # Costruttore
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        
        self.grad = 0.0
        self._backward = lambda: None   # Funzione che non fa nulla
        self._prev = set(_children) 
        self._op = _op
        self.label = label
        
    # Metodo utilizzato da Python per la visualizzazione dell'oggetto. Consente di impostare un visualizzazione user friendly
    def __repr__(self):
        return f"Value(data={self.data})"
    
    # Definisce l'operatore per la somma (+). Quando trova l'operatore +, Python chiama questo metodo
    def __add__(self, other):
        
        # Per gestire la somma di un Value con una costante
        other = other if isinstance(other, Value) else Value(other)
        
        out = Value(self.data + other.data, (self, other), '+')
        
        # Definisco una funzione che applica la chain rule per calcolare la derivata dei 2 termini rispetto all'output.
        # Dal momento che la local derivative per una somma è = 1, moltiplico 1 * la derivata del risultato della somma
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    
    # Metodo x negazione (-Value()). Utilizzato dal metodo successivo x sottrazione
    def __neg__(self):
        return self * -1
    
    # Definisce l'operatore sottrazione
    def __sub__(self, other):
        return self + (-other)
    
    
    # Definisce l'operatore per la prodottp (*). Quando trova l'operatore *, Python chiama questo metodo
    def __mul__(self, other):
        
         # Per gestire la somma di un Value con una costante
        other = other if isinstance(other, Value) else Value(other)
        
        out = Value(self.data * other.data, (self, other), '*')
        
        # Definisco una funzione che applica la chain rule per calcolare la derivata dei 2 fattori del prodotto
        # rispetto all'output.
        # Dal momento che la local derivative per un prodotto è = al valore dell'altro fattore,
        # moltiplico l'altro fattore * la derivata del prodotto
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    # Questo metodo viene richiamato da Python in caso di 2 * Value(). Restituirebbe un errore e quindi Python
    # richiama questo metodo per dare l'opportunità di gestire l'errore invertendo l'ordine degli operatori
    def __rmul__(self,other):
        return self * other
    
    def __radd__(self,other):
        return self + other
    
    # Questo metodo viene richiamato dabPython per l'elevazione a potenza (**n). Viene utilizzato per implementare la divisione
    # in quanto a / b corrisponde a a * (1/b) e quindi a * (b**-1)
    #
    def __pow__(self, other):
        assert isinstance(other, (int, float)), 'only supporting int/float powers for now'
        out = Value(self.data**other, (self,), f'**{other}')
        
        # Sappiamo che la derivata di una potenza è uguale a n x**(n-1)
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out
    
    
    # Metodo per la divisione: utilizza la moltiplicazione del valore del dividendo 
    # per il divisone elevato a potenza -1. Quindi utilizza il metodo precedente
    #
    def __truediv__(self,other):
        return self * other**-1
    
    # Metodo per il calcolo della funzione tanh. 
    # E' possibile utilizzare la funzione "composta" invece che le singole operazioni che la compongono in quanto 
    # siamo in grado di calcolarne la derivata
    #
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        
        # Definisco una funzione che applica la chain rule per calcolare la derivata della funzione tanh 
        # rispetto all'output.
        # Dal momento che la local derivative per tanh è = (1 - x**2), applicando la chain rule
        # moltiplico questa local derivative* la derivata del prodotto
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out
    
    # Calcolo dell'a funzione exp: costituisce una delle operazioni che fanno parte della formula di tanh
    # 
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward= _backward
        
        return out
        
        
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
    

In [6]:
class Neuron:
    
    # Definisco il costruttore che riceve in input il numero di input al Neuron
    def __init__(self, nin):
        
        # Inizializzo i weigths e bias sulla base del numero di inputs
        #
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
        
    # Metodo richiamato quando l'istanza della classe viene invocata passando il parametro previsto
    # il questo caso il metodo esegue il forward pass
    def __call__(self, x):
        # w * x +b
        # Nel nostro caso dovremo moltiplicare tutti gli elementi di w per tutti gli elementi di x (pairwise)
        # zip riceve 2 iterators e crea un iterator che tratte le tuple date dai 2 iterator
        act = sum(wi*xi for wi, xi in zip(self.w, x)) +self.b
        
        # Per ragioni di efficienza sarebbe preferibile
        # act = sum((wi*xi for wi, xi in zip(self.w,x)), self.b)
        out = act.tanh()
        return out
    
    # Creo una lista composta da tutti i weigths e dal bias (trasformato in list x consentire la somma di List + List = List)
    def parameters(self):
        return self.w + [self.b]
    
class Layer:
    
    # Il cotruttore riceve in input il numero di input e il numero di neurons che costituiscono il layer
    #
    def __init__(self, nin, nout):
        
        # Il layer è costituito da una lista di nout Neurons
        #
        self.neurons = [Neuron(nin) for _ in range(nout)]

    # Calcoliamo la lista degli output di ogni singolo neuron che compone il layer
    def __call__(self, x):
        
        outs = [n(x) for n in self.neurons]
        # return outs
        # Per gestire il fatto che può ritornare un singolo numero
        return outs[0] if len(outs) == 1 else outs

    # Creo una lista composta da tutti i weigths e dal bias di tutti i neuron che compongono il layer
    def parameters(self):
        # params = [neuron.parameters() for neuron in self.neurons]
        params = [p for neuron in self.neurons for p in neuron.parameters()]
        return params
    

class MLP:
    
    # Riceve il numero di inputs e la lista con le dimensioni dei layers che costutuiscono l'MLP
    def __init__(self, nin, nouts):
        
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
   
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)                                               
        return x    

    # Creo una lista composta da tutti i weigths e dal bias di tutti i neuron di tutti i layers che compongono il MLP
    def parameters(self):
        params = [p for layer in self.layers for p in layer.parameters()]
        return params

Definisco i datasets

In [66]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1,0, 1.0] # targets

Inizializzo l'MLP ed eseguo il forward pass per ottenere le prediction (basate sui valori di inizializzazione dei param)

In [67]:
n = MLP(3, [4, 4, 1])

ypred = [n(x) for x in xs]
ypred

[Value(data=0.5148315070801582),
 Value(data=0.06388505770520524),
 Value(data=0.45604447265991066),
 Value(data=0.5091537274871519)]

Dopo il foreward pass posso calcolare il valore della loss function

In [68]:
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss

Value(data=3.746542907108057)

Innesco il meccanismo di backprop al fine di calcolare i gradient

In [69]:
loss.backward()

A titolo di esempio visualizzo il valore di 1 parametro

In [70]:
n.layers[0].neurons[0].w[0].data

-0.8143905006996706

e visualizzo il corrispondente gradient

In [71]:
n.layers[0].neurons[0].w[0].grad

0.07828044382760115

In [72]:
len(n.parameters())

41

A questo punto è possibile iterare i parametri al fine di aggiustarli per minimizzare la loss function.
Per fare ciò è necessario aggiungere al valore del parametro per il valore del parametro stesso moltiplicato x un valore molto piccolo x il gradient (di segno contrario in quanto la loss function deve diminuire mantre il gradient indica la direzione dell'incremento 

In [73]:
for p in n.parameters():
    p.data += -0.1 * p.grad

In [74]:
n.layers[0].neurons[0].w[0].data

-0.8222185450824306

Posso rifare un forward pass e ricalcolare il loss che dovrebbe essersi ridotto rispetto a prima

In [75]:
ypred = [n(x) for x in xs]
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss

Value(data=2.1308389616573935)

In [76]:
loss.backward()

In [77]:
for p in n.parameters():
    p.data += -0.1 * p.grad

In [78]:
ypred = [n(x) for x in xs]
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss

Value(data=1.5751268934142049)

In [79]:
loss.backward()
for p in n.parameters():
    p.data += -0.1 * p.grad
ypred = [n(x) for x in xs]
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss

Value(data=2.493067183162297)

In [80]:
loss.backward()
for p in n.parameters():
    p.data += -0.1 * p.grad
ypred = [n(x) for x in xs]
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss

Value(data=0.588082593034952)

In [86]:
ypred

[Value(data=0.9968650419192825),
 Value(data=-0.9999999806140204),
 Value(data=-0.9999998300916838),
 Value(data=0.9957767757744413)]

Come si può vedere ora le prediction sono molto vicine ai targets (1.0, -1.0, -1.0, 1.0)

Vediamo ora come implementare il training loop

In [105]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1,0, 1.0] # targets

n = MLP(3, [4, 4, 1])


In [111]:
for k in range(20):
    
    # forward pass
    ypred = [n(x) for x in xs]
    
    # calcolo loss
    loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
    
    # backward pass (azzerando il gradient di tutti i parametri)
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    
    # update parameter
    for p in n.parameters():
        p.data += -0.05 * p.grad
    print(k, loss.data)

0 0.09861242677690318
1 0.15555810902052636
2 0.09748605759091127
3 0.14666298581357107
4 0.09218749146861113
5 0.13397104442866134
6 0.08853318271757107
7 0.12489930172494398
8 0.08503915074207803
9 0.11712653857268703
10 0.08198989990745767
11 0.11076779661812479
12 0.07934524095183296
13 0.10555794300845334
14 0.07707219132655405
15 0.10129138481563378
16 0.07512779835691485
17 0.09778915977201577
18 0.07346535609251173
19 0.09489853939007181


In [112]:
ypred

[Value(data=0.8889009846882733),
 Value(data=-0.9926469404321338),
 Value(data=-0.918060115664377),
 Value(data=0.275294998241691)]